# Importing data

In [1]:
import pandas as pd
true = pd.read_csv('../raw_data/True.csv')
fake = pd.read_csv('../raw_data/Fake.csv')

# Dropping some columns 

In [2]:
true.drop(columns = ['date', 'subject'], inplace = True)
fake.drop(columns = ['date', 'subject'], inplace = True)

# Creating new datasets so we dont mess with the real ones

In [3]:
true_copy = true.copy()
fake_copy = fake.copy()

# Brief Data Cleaning

In [4]:
stop_words = ['/Getty Images']
pat = '|'.join(r"\b{}\b".format(x) for x in stop_words)
fake_copy['text'] = fake_copy['text'].str.replace(pat, '')
true_copy['text'] = true_copy['text'].str.replace(pat, '')

# Adding features ratio

## Length Feature

In [5]:
true_copy['title_length_char'] = true_copy.title.str.len()

In [6]:
fake_copy['title_length_char'] = fake_copy.title.str.len()

## Punctuation ratio

## UpperCase Ratio

In [7]:
true_copy['title_Upper'] = true_copy['title'].str.count(r'[A-Z]')

fake_copy['title_Upper'] = fake_copy['title'].str.count(r'[A-Z]')


In [8]:
true_copy['title_lower_ratio'] = true_copy.title_Upper / true_copy.title_length_char
true_copy.drop(columns = 'title_Upper', inplace = True)

In [9]:
fake_copy['title_lower_ratio'] = fake_copy.title_Upper / fake_copy.title_length_char
fake_copy.drop(columns = 'title_Upper', inplace = True)

# Starting a model

## First create target and concat datasets

In [10]:
true_copy['score'] = 1
fake_copy['score'] = 0

In [11]:
data = pd.concat([true_copy, fake_copy], ignore_index=True)

In [12]:
data.head()

,title,text,title_length_char,title_lower_ratio,score
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,64,0.062500,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,64,0.062500,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,60,0.116667,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,59,0.135593,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,69,0.057971,1


## Let's initialize a model

In [13]:
import time
start = time.time()

In [14]:
x = data.drop(columns='score')
y = data['score']

In [15]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.3)

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer


preprocessor = ColumnTransformer([
    ('vectorizer_title', CountVectorizer(), 'title'),
    ('vectorizer_text', CountVectorizer(), 'text'),
    ('MinMaxScaler', MinMaxScaler(), ['title_length_char'])
])

final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('nb', MultinomialNB())])

In [17]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'preprocessing__vectorizer_title__ngram_range': ((2,2),(3,3)),
    'preprocessing__vectorizer_text__ngram_range': ((2,2),(3,3)),
    'preprocessing__vectorizer_text__max_df' : [0.8,1.0],
    'preprocessing__vectorizer_text__min_df' : [0.05,0.1],
    'nb__alpha': (2,3,4,5)}
grid_search = GridSearchCV(final_pipe,
                           parameters,
                           scoring = ["f1", "accuracy", "recall"],
                           refit= "accuracy",
                           cv=5,
                           verbose = 0)

In [18]:
start = time.time()
grid_search.fit(x_train,y_train)
stop = time.time()
print(f"Training time: {stop - start}s")

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fi

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fi

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fi

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fi

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fi

...

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fi

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fi

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fi

Training time: 56256.85180473328s


/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [20]:
tuned_pipe = grid_search.best_estimator_

In [22]:
tuned_pipe

Pipeline(memory=None,
     steps=[('preprocessing', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('vectorizer_title', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', ...0, 1)), ['title_length_char'])])), ('nb', MultinomialNB(alpha=2, class_prior=None, fit_prior=True))])

In [21]:
grid_search.best_params_

{'nb__alpha': 2,
 'preprocessing__vectorizer_text__max_df': 0.8,
 'preprocessing__vectorizer_text__min_df': 0.05,
 'preprocessing__vectorizer_text__ngram_range': (2, 2),
 'preprocessing__vectorizer_title__ngram_range': (2, 2)}

In [23]:
pd.DataFrame(grid_search.cv_results_)

/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/andreamontagnoli/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_nb__alpha,param_preprocessing__vectorizer_text__max_df,param_preprocessing__vectorizer_text__min_df,param_preprocessing__vectorizer_text__ngram_range,param_preprocessing__vectorizer_title__ngram_range,params,...,mean_test_recall,std_test_recall,rank_test_recall,split0_train_recall,split1_train_recall,split2_train_recall,split3_train_recall,split4_train_recall,mean_train_recall,std_train_recall
0,26.040344,0.256291,9.591013,0.160494,2,0.8,0.05,"(2, 2)","(2, 2)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.981353,0.001133,1,0.991425,0.991842,0.992009,0.992009,0.991343,0.991725,0.000287
1,26.091720,0.190174,9.420439,0.087643,2,0.8,0.05,"(2, 2)","(3, 3)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.975027,0.002622,7,0.991508,0.991509,0.991426,0.991176,0.991259,0.991376,0.000135
2,48.029550,2.765555,10.063296,0.259418,2,0.8,0.05,"(3, 3)","(2, 2)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.929075,0.005073,33,0.995754,0.994672,0.995172,0.994339,0.995505,0.995089,0.000521
3,45.316134,0.152430,9.742338,0.057817,2,0.8,0.05,"(3, 3)","(3, 3)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.803077,0.009059,49,0.999750,0.999834,0.999750,0.999834,0.999917,0.999817,0.000062
4,27.412513,1.269996,9.912058,0.329618,2,0.8,0.1,"(2, 2)","(2, 2)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.975493,0.003528,5,0.993590,0.993091,0.993507,0.993507,0.993507,0.993440,0.000178
5,27.214231,0.869638,9.966987,0.300224,2,0.8,0.1,"(2, 2)","(3, 3)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.963306,0.006303,19,0.993173,0.993257,0.993174,0.992841,0.992758,0.993041,0.000201
6,48.896401,1.725238,10.448598,0.418617,2,0.8,0.1,"(3, 3)","(2, 2)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.905567,0.008289,41,0.994006,0.993008,0.994256,0.992841,0.993757,0.993574,0.000556
7,47.938042,0.985021,9.998568,0.192319,2,0.8,0.1,"(3, 3)","(3, 3)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.717768,0.007178,57,0.999833,0.999834,0.999917,0.999750,0.999917,0.999850,0.000062
8,26.775563,0.391529,9.829521,0.114975,2,1,0.05,"(2, 2)","(2, 2)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.981353,0.001133,1,0.991425,0.991842,0.992009,0.992009,0.991343,0.991725,0.000287
9,26.822918,0.441557,9.786222,0.299613,2,1,0.05,"(2, 2)","(3, 3)","{'nb__alpha': 2, 'preprocessing__vectorizer_te...",...,0.975027,0.002622,7,0.991508,0.991509,0.991426,0.991176,0.991259,0.991376,0.000135


In [25]:
tuned_pipe.score(x_test,y_test)

0.9723088344469191

In [24]:
results = pd.DataFrame(grid_search.cv_results_)
results.iloc[0]

mean_fit_time                                                                                   26.0403
std_fit_time                                                                                   0.256291
mean_score_time                                                                                 9.59101
std_score_time                                                                                 0.160494
param_nb__alpha                                                                                       2
param_preprocessing__vectorizer_text__max_df                                                        0.8
param_preprocessing__vectorizer_text__min_df                                                       0.05
param_preprocessing__vectorizer_text__ngram_range                                                (2, 2)
param_preprocessing__vectorizer_title__ngram_range                                               (2, 2)
params                                                {'nb__alph

In [ ]:
import pickle

# Export pipeline as pickle file
with open("pipeline.pkl", "wb") as file:
    pickle.dump(tuned_pipe, file )